# 07. PyTorch Experiment Tracking Exercise Template

Welcome to the 07. PyTorch Experiment Tracking exercise template notebook.

> **Note:** There may be more than one solution to each of the exercises. This notebook only shows one possible example.

## Resources

1. These exercises/solutions are based on [section 07. PyTorch Transfer Learning](https://www.learnpytorch.io/07_pytorch_experiment_tracking/) of the Learn PyTorch for Deep Learning course by Zero to Mastery.
2. See a live [walkthrough of the solutions (errors and all) on YouTube](https://youtu.be/cO_r2FYcAjU).
3. See [other solutions on the course GitHub](https://github.com/mrdbourke/pytorch-deep-learning/tree/main/extras/solutions).

> **Note:** The first section of this notebook is dedicated to getting various helper functions and datasets used for the exercises. The exercises start at the heading "Exercise 1: ...".

### Get various imports and helper functions

We'll need to make sure we have `torch` v.1.12+ and `torchvision` v0.13+.

In [2]:
# For this notebook to run with updated APIs, we need torch 1.12+ and torchvision 0.13+
try:
    import torch
    import torchvision

    major = int(torch.__version__.split(".")[0])
    minor = int(torch.__version__.split(".")[1])
    assert (major >= 2) or (minor >= 12), "torch version should be 1.12+"
    assert int(torchvision.__version__.split(".")[1]) >= 13, "torchvision version should be 0.13+"
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")
except:
    print(f"[INFO] torch/torchvision versions not as required, installing nightly versions.")
    !pip3 install -U torch torchvision torchaudio
    import torch
    import torchvision
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")

torch version: 2.1.0+cu121
torchvision version: 0.16.0+cu121


In [3]:
 # Make sure we have a GPU
 device = "cuda" if torch.cuda.is_available() else "cpu"
 device

'cuda'

In [4]:
# Get regular imports
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

# Try to import the going_modular directory, download it from GitHub if it doesn't work
try:
    from going_modular import data_setup, engine
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !rm -rf pytorch-deep-learning
    from going_modular.going_modular import data_setup, engine

In [5]:
# Set seeds
def set_seeds(seed: int=42):
    """Sets random sets for torch operations.

    Args:
        seed (int, optional): Random seed to set. Defaults to 42.
    """
    # Set the seed for general torch operations
    torch.manual_seed(seed)
    # Set the seed for CUDA torch operations (ones that happen on the GPU)
    torch.cuda.manual_seed(seed)

In [6]:
import os
import zipfile

from pathlib import Path

import requests

def download_data(source: str,
                  destination: str,
                  remove_source: bool = True) -> Path:
    """Downloads a zipped dataset from source and unzips to destination.

    Args:
        source (str): A link to a zipped file containing data.
        destination (str): A target directory to unzip data to.
        remove_source (bool): Whether to remove the source after downloading and extracting.

    Returns:
        pathlib.Path to downloaded data.

    Example usage:
        download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                      destination="pizza_steak_sushi")
    """
    # Setup path to data folder
    data_path = Path("data/")
    image_path = data_path / destination

    # If the image folder doesn't exist, download it and prepare it...
    if image_path.is_dir():
        print(f"[INFO] {image_path} directory exists, skipping download.")
    else:
        print(f"[INFO] Did not find {image_path} directory, creating one...")
        image_path.mkdir(parents=True, exist_ok=True)

        # Download pizza, steak, sushi data
        target_file = Path(source).name
        with open(data_path / target_file, "wb") as f:
            request = requests.get(source)
            print(f"[INFO] Downloading {target_file} from {source}...")
            f.write(request.content)

        # Unzip pizza, steak, sushi data
        with zipfile.ZipFile(data_path / target_file, "r") as zip_ref:
            print(f"[INFO] Unzipping {target_file} data...")
            zip_ref.extractall(image_path)

        # Remove .zip file
        if remove_source:
            os.remove(data_path / target_file)

    return image_path

image_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                           destination="pizza_steak_sushi")
image_path

[INFO] data/pizza_steak_sushi directory exists, skipping download.


PosixPath('data/pizza_steak_sushi')

In [7]:
from torch.utils.tensorboard import SummaryWriter
def create_writer(experiment_name: str,
                  model_name: str,
                  extra: str=None):
    """Creates a torch.utils.tensorboard.writer.SummaryWriter() instance saving to a specific log_dir.

    log_dir is a combination of runs/timestamp/experiment_name/model_name/extra.

    Where timestamp is the current date in YYYY-MM-DD format.

    Args:
        experiment_name (str): Name of experiment.
        model_name (str): Name of model.
        extra (str, optional): Anything extra to add to the directory. Defaults to None.

    Returns:
        torch.utils.tensorboard.writer.SummaryWriter(): Instance of a writer saving to log_dir.

    Example usage:
        # Create a writer saving to "runs/2022-06-04/data_10_percent/effnetb2/5_epochs/"
        writer = create_writer(experiment_name="data_10_percent",
                               model_name="effnetb2",
                               extra="5_epochs")
        # The above is the same as:
        writer = SummaryWriter(log_dir="runs/2022-06-04/data_10_percent/effnetb2/5_epochs/")
    """
    from datetime import datetime
    import os

    # Get timestamp of current date (all experiments on certain day live in same folder)
    timestamp = datetime.now().strftime("%Y-%m-%d") # returns current date in YYYY-MM-DD format

    if extra:
        # Create log directory path
        log_dir = os.path.join("runs", timestamp, experiment_name, model_name, extra)
    else:
        log_dir = os.path.join("runs", timestamp, experiment_name, model_name)

    print(f"[INFO] Created SummaryWriter, saving to: {log_dir}...")
    return SummaryWriter(log_dir=log_dir)

In [8]:
# Create a test writer
writer = create_writer(experiment_name="test_experiment_name",
                       model_name="this_is_the_model_name",
                       extra="add_a_little_extra_if_you_want")

[INFO] Created SummaryWriter, saving to: runs/2023-10-25/test_experiment_name/this_is_the_model_name/add_a_little_extra_if_you_want...


In [9]:
from typing import Dict, List
from tqdm.auto import tqdm

from going_modular.engine import train_step, test_step

# Add writer parameter to train()
def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device, 
          writer: torch.utils.tensorboard.writer.SummaryWriter # new parameter to take in a writer
          ) -> Dict[str, List]:
    """Trains and tests a PyTorch model.

    Passes a target PyTorch models through train_step() and test_step()
    functions for a number of epochs, training and testing the model
    in the same epoch loop.

    Calculates, prints and stores evaluation metrics throughout.

    Stores metrics to specified writer log_dir if present.

    Args:
      model: A PyTorch model to be trained and tested.
      train_dataloader: A DataLoader instance for the model to be trained on.
      test_dataloader: A DataLoader instance for the model to be tested on.
      optimizer: A PyTorch optimizer to help minimize the loss function.
      loss_fn: A PyTorch loss function to calculate loss on both datasets.
      epochs: An integer indicating how many epochs to train for.
      device: A target device to compute on (e.g. "cuda" or "cpu").
      writer: A SummaryWriter() instance to log model results to.

    Returns:
      A dictionary of training and testing loss as well as training and
      testing accuracy metrics. Each metric has a value in a list for 
      each epoch.
      In the form: {train_loss: [...],
                train_acc: [...],
                test_loss: [...],
                test_acc: [...]} 
      For example if training for epochs=2: 
              {train_loss: [2.0616, 1.0537],
                train_acc: [0.3945, 0.3945],
                test_loss: [1.2641, 1.5706],
                test_acc: [0.3400, 0.2973]} 
    """
    # Create empty results dictionary
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }

    # Loop through training and testing steps for a number of epochs
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.75)
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
        test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)

        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

        # Step the scheduler
        scheduler.step()

        ### New: Use the writer parameter to track experiments ###
        # See if there's a writer, if so, log to it
        if writer:
            # Add results to SummaryWriter
            writer.add_scalars(main_tag="Loss", 
                               tag_scalar_dict={"train_loss": train_loss,
                                                "test_loss": test_loss},
                               global_step=epoch)
            writer.add_scalars(main_tag="Accuracy", 
                               tag_scalar_dict={"train_acc": train_acc,
                                                "test_acc": test_acc}, 
                               global_step=epoch)

            # Close the writer
            writer.close()
        else:
            pass
    ### End new ###
        
    # Return the filled results at the end of the epochs
    return results

### Download data

Using the same data from https://www.learnpytorch.io/07_pytorch_experiment_tracking/

In [10]:
# Download 10 percent and 20 percent training data (if necessary)
data_10_percent_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                                     destination="pizza_steak_sushi")

data_20_percent_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip",
                                     destination="pizza_steak_sushi_20_percent")

[INFO] data/pizza_steak_sushi directory exists, skipping download.
[INFO] data/pizza_steak_sushi_20_percent directory exists, skipping download.


In [11]:
# Setup training directory paths
train_dir_10_percent = data_10_percent_path / "train"
train_dir_20_percent = data_20_percent_path / "train"

# Setup testing directory paths (note: use the same test dataset for both to compare the results)
test_dir = data_20_percent_path / "test"

# Check the directories
print(f"Training directory 10%: {train_dir_10_percent}")
print(f"Training directory 20%: {train_dir_20_percent}")
print(f"Testing directory: {test_dir}")

Training directory 10%: data/pizza_steak_sushi/train
Training directory 20%: data/pizza_steak_sushi_20_percent/train
Testing directory: data/pizza_steak_sushi_20_percent/test


In [12]:
from torchvision import transforms

# Create a transform to normalize data distribution to be inline with ImageNet
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], # values per colour channel [red, green, blue]
                                 std=[0.229, 0.224, 0.225])

# Create a transform pipeline
simple_transform = transforms.Compose([
                                       transforms.Resize((224, 224)),
                                       transforms.ToTensor(), # get image values between 0 & 1
                                       normalize
])

### Turn data into DataLoaders

In [13]:
BATCH_SIZE = 32

# Create 10% training and test DataLoaders
train_dataloader_10_percent, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir_10_percent,
                                                                                          test_dir=test_dir,
                                                                                          transform=simple_transform,
                                                                                          batch_size=BATCH_SIZE)

# Create 20% training and test DataLoaders
train_dataloader_20_percent, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir_20_percent,
                                                                                          test_dir=test_dir,
                                                                                          transform=simple_transform,
                                                                                          batch_size=BATCH_SIZE)

# Find the number of samples/batches per dataloader (using the same test_dataloader for both experiments)
print(f"Number of batches of size {BATCH_SIZE} in 10 percent training data: {len(train_dataloader_10_percent)}")
print(f"Number of batches of size {BATCH_SIZE} in 20 percent training data: {len(train_dataloader_20_percent)}")
print(f"Number of batches of size {BATCH_SIZE} in testing data: {len(train_dataloader_10_percent)} (all experiments will use the same test set)")
print(f"Number of classes: {len(class_names)}, class names: {class_names}")

Number of batches of size 32 in 10 percent training data: 8
Number of batches of size 32 in 20 percent training data: 15
Number of batches of size 32 in testing data: 8 (all experiments will use the same test set)
Number of classes: 3, class names: ['pizza', 'steak', 'sushi']


## Exercise 1: Pick a larger model from [`torchvision.models`](https://pytorch.org/vision/main/models.html) to add to the list of experiments (for example, EffNetB3 or higher)

* How does it perform compared to our existing models?
* **Hint:** You'll need to set up an exerpiment similar to [07. PyTorch Experiment Tracking section 7.6](https://www.learnpytorch.io/07_pytorch_experiment_tracking/#76-create-experiments-and-set-up-training-code).

In [14]:
#Set up a list of models we'll loop through
models = ["effnetb0", "effnetb2", "effnetb3"]

epochs = [5, 10]

# Set up a list of dataloaders we will go through
train_dataloaders = {"data_10_percent": train_dataloader_10_percent,
                     "data_20_percent": train_dataloader_20_percent}

In [15]:
torchvision.models.efficientnet_b3().classifier

Sequential(
  (0): Dropout(p=0.3, inplace=True)
  (1): Linear(in_features=1536, out_features=1000, bias=True)
)

In [16]:
OUT_FEATURES = len(class_names)

def create_effnetb0():
    model = torchvision.models.efficientnet_b0(weights='DEFAULT').to(device)

    # Freeze the base layer
    for param in model.features.parameters():
        param.requires_grad = False


    # Change the classifier head
    model.classifier = nn.Sequential(
        nn.Dropout(p=0.2),
        nn.Linear(in_features=1280, out_features=OUT_FEATURES)
    ).to(device)
    
    model.name = "effnetb0"
    print(f"[INFO] Created new {model.name} model.")
    return model

def create_effnetb2():
    model = torchvision.models.efficientnet_b2(weights='DEFAULT').to(device)

    # Freeze the base layer
    for param in model.features.parameters():
        param.requires_grad = False


    # Change the classifier head
    model.classifier = nn.Sequential(
        nn.Dropout(p=0.3),
        nn.Linear(in_features=1408, out_features=OUT_FEATURES)
    ).to(device)
    
    model.name = "effnetb2"
    print(f"[INFO] Created new {model.name} model.")
    return model

def create_effnetb3():
    model = torchvision.models.efficientnet_b3(weights='DEFAULT').to(device)

    # Freeze the base layer
    for param in model.features.parameters():
        param.requires_grad = False


    # Change the classifier head
    model.classifier = nn.Sequential(
        nn.Dropout(p=0.3),
        nn.Linear(in_features=1536, out_features=OUT_FEATURES)
    ).to(device)
    
    model.name = "effnetb3"
    print(f"[INFO] Created new {model.name} model.")
    return model

In [17]:
# Override the get_state_dict method to skip has check
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from torchvision.models._api import WeightsEnum
from torch.hub import load_state_dict_from_url

def get_state_dict(self, *args, **kwargs):
    kwargs.pop("check_hash")
    return load_state_dict_from_url(self.url, *args, **kwargs)
WeightsEnum.get_state_dict = get_state_dict

In [18]:
from going_modular.utils import save_model
from colorama import Fore, Style, init



def create_and_train(models=List[str],
                     epochs=List[int],
                     data_loaders=Dict):
    init(autoreset=True)
    experiment_number = 0
    
    for dataloader_name, train_dataloader in data_loaders.items():
        for epoch in epochs:
            for model_name in models:
                experiment_number +=1
                print(Style.BRIGHT + Fore.RED + f"Experiment Number: {experiment_number}")
                print(f"Model: {model_name}")
                print(f"DataLoader: {dataloader_name}")
                print(f"Number of epochs: {epoch}")
        
                try:
                    assert model_name in ["effnetb0", "effnetb2", "effnetb3"], "Error: Model unrecognized, defaulting to Effnetb0"
        
                    if model_name == "effnetb0":
                        model = create_effnetb0() 
                    elif model_name == "effnetb2":
                        model = create_effnetb2()
                    else:
                        model = create_effnetb3()
                except AssertionError as e:
                    e = Style.BRIGHT + Fore.RED + e
                    print(e)
                    model = create_effnetb0()
        
                # Create optimizer and loss functions
                loss_fn = nn.CrossEntropyLoss()
                optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
                
                train(model=model,
                      train_dataloader=train_dataloader,
                      test_dataloader=test_dataloader,
                      optimizer=optimizer,
                      loss_fn=loss_fn,
                      epochs=epoch,
                      device=device,
                      writer=create_writer(experiment_name=dataloader_name,
                                           model_name=model_name,
                                           extra=f"{epoch}_epochs"))
        
                # Save the model
                save_filepath = f"07_{model_name}_{dataloader_name}_{epochs}.pth"
                save_model(model=model,
                           target_dir="models",
                           model_name=save_filepath)
                print(Style.BRIGHT + Fore.GREEN + "-"*75 + "\n")

## Exercise 2. Introduce data augmentation to the list of experiments using the 20% pizza, steak, sushi training and test datasets, does this change anything?
    
* For example, you could have one training DataLoader that uses data augmentation (e.g. `train_dataloader_20_percent_aug` and `train_dataloader_20_percent_no_aug`) and then compare the results of two of the same model types training on these two DataLoaders.
* **Note:** You may need to alter the `create_dataloaders()` function to be able to take a transform for the training data and the testing data (because you don't need to perform data augmentation on the test data). See [04. PyTorch Custom Datasets section 6](https://www.learnpytorch.io/04_pytorch_custom_datasets/#6-other-forms-of-transforms-data-augmentation) for examples of using data augmentation or the script below for an example:

```python
# Note: Data augmentation transform like this should only be performed on training data
train_transform_data_aug = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.TrivialAugmentWide(),
    transforms.ToTensor(),
    normalize
])

# Create a helper function to visualize different augmented (and not augmented) images
def view_dataloader_images(dataloader, n=10):
    if n > 10:
        print(f"Having n higher than 10 will create messy plots, lowering to 10.")
        n = 10
    imgs, labels = next(iter(dataloader))
    plt.figure(figsize=(16, 8))
    for i in range(n):
        # Min max scale the image for display purposes
        targ_image = imgs[i]
        sample_min, sample_max = targ_image.min(), targ_image.max()
        sample_scaled = (targ_image - sample_min)/(sample_max - sample_min)

        # Plot images with appropriate axes information
        plt.subplot(1, 10, i+1)
        plt.imshow(sample_scaled.permute(1, 2, 0)) # resize for Matplotlib requirements
        plt.title(class_names[labels[i]])
        plt.axis(False)

# Have to update `create_dataloaders()` to handle different augmentations
import os
from torch.utils.data import DataLoader
from torchvision import datasets

NUM_WORKERS = os.cpu_count() # use maximum number of CPUs for workers to load data

# Note: this is an update version of data_setup.create_dataloaders to handle
# differnt train and test transforms.
def create_dataloaders(
    train_dir,
    test_dir,
    train_transform, # add parameter for train transform (transforms on train dataset)
    test_transform,  # add parameter for test transform (transforms on test dataset)
    batch_size=32, num_workers=NUM_WORKERS
):
    # Use ImageFolder to create dataset(s)
    train_data = datasets.ImageFolder(train_dir, transform=train_transform)
    test_data = datasets.ImageFolder(test_dir, transform=test_transform)

    # Get class names
    class_names = train_data.classes

    # Turn images into data loaders
    train_dataloader = DataLoader(
        train_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
    )
    test_dataloader = DataLoader(
        test_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
    )

    return train_dataloader, test_dataloader, class_names
```

In [23]:
# Modify create_dataloader to accept two transforms

"""
Contains funcitonality for creating PyTorch DataLoader's for
image classification data.
"""
import os

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    train_transform: transforms.Compose,
    test_transform: transforms.Compose=torchvision.models.EfficientNet_B0_Weights.DEFAULT.transforms(),
    batch_size: int=32,
    num_workers: int=NUM_WORKERS
):
    """Create training and testing Dataloaders

    Takes in a training and testing directory path and turns them into 
    PyTorch Datasets and then into DataLoaders

    Args:
        train_dir: Path to training directory.
        test_dir: Path to testing directory.
        train_transform: torchvision transform to perform on training.
        test_transform: torchvision transform to be performed on test data, default is efficientnetb0's transform
        batch_size: Number of samples per batch in each of the DataLoaders.
        num_workers: An integer for number of workers per DataLoader.

    Returns:
        A tuple of (train_dataloader, test_dataloader, class_names).
        Where class_names is a list of the target classes.
        Example usage:
    """

    tensor_transform = transforms.ToTensor()
    
    # Create the datasets, making sure to apply the tensor transform before any other transforms
    train_data = datasets.ImageFolder(root=train_dir,
                                      transform=transforms.Compose([tensor_transform, train_transform]),
                                      target_transform=None)
    
    test_data = datasets.ImageFolder(root=test_dir,
                                     transform=transforms.Compose([tensor_transform, test_transform]),
                                     target_transform=None)
    class_names = train_data.classes

    
    train_dataloader = DataLoader(dataset=train_data,
                                  batch_size=batch_size,
                                  shuffle=True,
                                  num_workers=num_workers,
                                  pin_memory=True)

    test_dataloader = DataLoader(dataset=test_data,
                                 batch_size=batch_size,
                                 shuffle=False,
                                 num_workers=num_workers,
                                 pin_memory=True)
    
    return train_dataloader, test_dataloader, class_names

In [24]:
from torchvision.transforms import v2
# Create a data augmentation transform using PyTorch's new v2 transforms
train_transform = v2.Compose([
    v2.ToDtype(torch.float32, scale=True),
    v2.RandomResizedCrop(size=(224, 224), antialias=True),
    v2.RandomHorizontalFlip(p=0.5),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create an augmented train dataloader and regular test transform
train_dataloader_20_percent_augmented, test_dataloader_20_percent, class_names = create_dataloaders(train_dir=train_dir_20_percent,
                                                           test_dir=test_dir,
                                                           train_transform=train_transform,
                                                           test_transform=torchvision.models.EfficientNet_B0_Weights.DEFAULT.transforms())

In [25]:
train_dataloaders_augmented = {"data_20_percent_augmented": train_dataloader_20_percent_augmented,
                               "data_20_percent": train_dataloader_20_percent}

In [26]:
%%time

create_and_train(models=models,
                 epochs=epochs,
                 data_loaders=train_dataloaders_augmented)

Experiment Number: 1
Model: effnetb0
DataLoader: data_20_percent_augmented
Number of epochs: 5
[INFO] Created new effnetb0 model.
[INFO] Created SummaryWriter, saving to: runs/2023-10-25/data_20_percent_augmented/effnetb0/5_epochs...


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0210 | train_acc: 0.5250 | test_loss: 0.6855 | test_acc: 0.8562
Epoch: 2 | train_loss: 0.7472 | train_acc: 0.8250 | test_loss: 0.5637 | test_acc: 0.9102
Epoch: 3 | train_loss: 0.6180 | train_acc: 0.8479 | test_loss: 0.4706 | test_acc: 0.9006
Epoch: 4 | train_loss: 0.5871 | train_acc: 0.8438 | test_loss: 0.4106 | test_acc: 0.9131
Epoch: 5 | train_loss: 0.5115 | train_acc: 0.8583 | test_loss: 0.3942 | test_acc: 0.9131
[INFO] Saving model to: models/07_effnetb0_data_20_percent_augmented_[5, 10].pth
---------------------------------------------------------------------------

Experiment Number: 2
Model: effnetb2
DataLoader: data_20_percent_augmented
Number of epochs: 5
[INFO] Created new effnetb2 model.
[INFO] Created SummaryWriter, saving to: runs/2023-10-25/data_20_percent_augmented/effnetb2/5_epochs...


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0144 | train_acc: 0.5229 | test_loss: 0.8208 | test_acc: 0.7869
Epoch: 2 | train_loss: 0.7980 | train_acc: 0.7000 | test_loss: 0.6738 | test_acc: 0.8449
Epoch: 3 | train_loss: 0.6715 | train_acc: 0.8083 | test_loss: 0.5793 | test_acc: 0.8852
Epoch: 4 | train_loss: 0.6394 | train_acc: 0.7542 | test_loss: 0.5159 | test_acc: 0.9159
Epoch: 5 | train_loss: 0.5503 | train_acc: 0.8396 | test_loss: 0.4934 | test_acc: 0.8824
[INFO] Saving model to: models/07_effnetb2_data_20_percent_augmented_[5, 10].pth
---------------------------------------------------------------------------

Experiment Number: 3
Model: effnetb3
DataLoader: data_20_percent_augmented
Number of epochs: 5
[INFO] Created new effnetb3 model.
[INFO] Created SummaryWriter, saving to: runs/2023-10-25/data_20_percent_augmented/effnetb3/5_epochs...


...

Epoch: 1 | train_loss: 0.9221 | train_acc: 0.6312 | test_loss: 0.6199 | test_acc: 0.9097
Epoch: 2 | train_loss: 0.6834 | train_acc: 0.8042 | test_loss: 0.5188 | test_acc: 0.8972
Epoch: 3 | train_loss: 0.5273 | train_acc: 0.8667 | test_loss: 0.4387 | test_acc: 0.9034
Epoch: 4 | train_loss: 0.4565 | train_acc: 0.9062 | test_loss: 0.4177 | test_acc: 0.9068
Epoch: 5 | train_loss: 0.4704 | train_acc: 0.8625 | test_loss: 0.3766 | test_acc: 0.9131
Epoch: 6 | train_loss: 0.4397 | train_acc: 0.8708 | test_loss: 0.3585 | test_acc: 0.9131
Epoch: 7 | train_loss: 0.3698 | train_acc: 0.9083 | test_loss: 0.3516 | test_acc: 0.9068
Epoch: 8 | train_loss: 0.3451 | train_acc: 0.9250 | test_loss: 0.3153 | test_acc: 0.9159
Epoch: 9 | train_loss: 0.3330 | train_acc: 0.9292 | test_loss: 0.2995 | test_acc: 0.9409
Epoch: 10 | train_loss: 0.3769 | train_acc: 0.8562 | test_loss: 0.3109 | test_acc: 0.9284
[INFO] Saving model to: models/07_effnetb0_data_20_percent_[5, 10].pth
--------------------------------------

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.9910 | train_acc: 0.5583 | test_loss: 0.7945 | test_acc: 0.8545
Epoch: 2 | train_loss: 0.7317 | train_acc: 0.8521 | test_loss: 0.6343 | test_acc: 0.8665
Epoch: 3 | train_loss: 0.5862 | train_acc: 0.8542 | test_loss: 0.5694 | test_acc: 0.9102
Epoch: 4 | train_loss: 0.4894 | train_acc: 0.9146 | test_loss: 0.5190 | test_acc: 0.8977
Epoch: 5 | train_loss: 0.4465 | train_acc: 0.9042 | test_loss: 0.4743 | test_acc: 0.9040
Epoch: 6 | train_loss: 0.4213 | train_acc: 0.9021 | test_loss: 0.4532 | test_acc: 0.9102
Epoch: 7 | train_loss: 0.3883 | train_acc: 0.9125 | test_loss: 0.4343 | test_acc: 0.8886
Epoch: 8 | train_loss: 0.3672 | train_acc: 0.9042 | test_loss: 0.4265 | test_acc: 0.9011
Epoch: 9 | train_loss: 0.3703 | train_acc: 0.8958 | test_loss: 0.4077 | test_acc: 0.8761
Epoch: 10 | train_loss: 0.3361 | train_acc: 0.9479 | test_loss: 0.4002 | test_acc: 0.8886
[INFO] Saving model to: models/07_effnetb2_data_20_percent_[5, 10].pth
--------------------------------------

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.9895 | train_acc: 0.5646 | test_loss: 0.8040 | test_acc: 0.8824
Epoch: 2 | train_loss: 0.7518 | train_acc: 0.7625 | test_loss: 0.6239 | test_acc: 0.9324
Epoch: 3 | train_loss: 0.5758 | train_acc: 0.9042 | test_loss: 0.5383 | test_acc: 0.8761
Epoch: 4 | train_loss: 0.5028 | train_acc: 0.9062 | test_loss: 0.4816 | test_acc: 0.9074
Epoch: 5 | train_loss: 0.4783 | train_acc: 0.8812 | test_loss: 0.4439 | test_acc: 0.9045
Epoch: 6 | train_loss: 0.5013 | train_acc: 0.8167 | test_loss: 0.4222 | test_acc: 0.8920
Epoch: 7 | train_loss: 0.4372 | train_acc: 0.8604 | test_loss: 0.3898 | test_acc: 0.9261
Epoch: 8 | train_loss: 0.3800 | train_acc: 0.9125 | test_loss: 0.3842 | test_acc: 0.9386
Epoch: 9 | train_loss: 0.4204 | train_acc: 0.8750 | test_loss: 0.3709 | test_acc: 0.9324
Epoch: 10 | train_loss: 0.3918 | train_acc: 0.8625 | test_loss: 0.3643 | test_acc: 0.9136
[INFO] Saving model to: models/07_effnetb3_data_20_percent_[5, 10].pth
--------------------------------------

## Exercise 3. Scale up the dataset to turn FoodVision Mini into FoodVision Big using the entire [Food101 dataset from `torchvision.models`](https://pytorch.org/vision/stable/generated/torchvision.datasets.Food101.html#torchvision.datasets.Food101)
    
* You could take the best performing model from your various experiments or even the EffNetB2 feature extractor we created in this notebook and see how it goes fitting for 5 epochs on all of Food101.
* If you try more than one model, it would be good to have the model's results tracked.
* If you load the Food101 dataset from `torchvision.models`, you'll have to create PyTorch DataLoaders to use it in training.
* **Note:** Due to the larger amount of data in Food101 compared to our pizza, steak, sushi dataset, this model will take longer to train.

In [28]:
# Create transforms we will use
transform_v2m = torchvision.models.EfficientNet_V2_M_Weights.DEFAULT.transforms()

# Download the data
train_data = torchvision.datasets.Food101(root="data",
                                          split="train",
                                          transform=transform_v2m,
                                          download=False)

test_data = torchvision.datasets.Food101(root="data",
                                          split="test",
                                          transform=transform_v2m,
                                          download=False)

100%|█████████████████████| 4996278331/4996278331 [07:51<00:00, 10595367.98it/s]


Extracting data/food-101.tar.gz to data


In [33]:
transform_v2m

ImageClassification(
    crop_size=[480]
    resize_size=[480]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [36]:
OUT_FEATURES = len(train_data.classes)

# Create a model
model_v2m = torchvision.models.efficientnet_v2_m(weights='DEFAULT').to(device)

# Freeze the base layer
for param in model_v2m.features.parameters():
    param.requires_grad = False

# Change the classifier head of the model
model_v2m.classifier = nn.Sequential(
    nn.Dropout(p=0.3, inplace=True),
    nn.Linear(in_features=1280, out_features=OUT_FEATURES)
).to(device)

In [ ]:
from torchinfo import summary


summary(model=model_v2m,
        input_size=(1, 3, 480, 480),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

In [40]:
# Create dataloaders
import os

train_dataloader_big = torch.utils.data.DataLoader(dataset=train_data,
                                                   shuffle=True,
                                                   pin_memory=True,
                                                   batch_size=BATCH_SIZE,
                                                   num_workers=os.cpu_count())

test_dataloader_big = torch.utils.data.DataLoader(dataset=test_data,
                                                   shuffle=False,
                                                   pin_memory=True,
                                                   batch_size=BATCH_SIZE,
                                                   num_workers=os.cpu_count())

In [42]:
foodvision_big_results = train(model=model_v2m,
                               train_dataloader=train_dataloader_big,
                               test_dataloader=test_dataloader_big,
                               optimizer=torch.optim.Adam(params=model_v2m.parameters(), lr=0.001),
                               loss_fn=torch.nn.CrossEntropyLoss(),
                               epochs=5,
                               device=device,
                               writer=create_writer(experiment_name="food101_all_data",
                                                    model_name="foodvision_big",
                                                    extra=f"{epochs}_epochs"))

[INFO] Created SummaryWriter, saving to: runs/2023-10-25/food101_all_data/foodvision_big/[5, 10]_epochs...


  0%|          | 0/5 [00:00<?, ?it/s]

KeyboardInterrupt: 